#### Firstly, import the necessary libraries.

In [2]:
import os
import json
import pandas as pd
import traceback

In [3]:
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain, SequentialChain
from langchain.callbacks import get_openai_callback
import PyPDF2

In [4]:
from langchain.chat_models import ChatOpenAI

#### Secondly, declare and load OpenAI API key, preparing it for utilization.

#### **NOTE!!!** There is an indirect way of OpenAI API key declaration...
Below is shown an alternative way to indirectly deploy the OpenAI API Key, following the steps below:

1. Create `.env` file where you will have to put your OpenAI API Key. i.e., `OPENAI_API_KEY="Add your key here"`

2. Import `load_dotenv` and then execute `load_dotenv()`.

In [5]:
from dotenv import load_dotenv

load_dotenv()

True

3. Declare `KEY`. You can also print it and check whether itfunctions properly.

In [6]:
KEY=os.getenv("OPENAI_API_KEY")
# print(KEY)

**REMEMBER** to insert `.env` into your .gitignore file!

#### `OpenAI()` vs. `ChatOpenAI()`:
The difference between `OpenAI()` and `ChatOpenAI()` in LangChain (a framework for working with LLMs) is:

- `OpenAI()` → Used for completion-based models (like GPT-3 text completion).
- `ChatOpenAI()` → Used for chat-based models (like GPT-4 and GPT-3.5-turbo), which work with structured messages.

In short:
- Use `OpenAI()` for single-turn completions.
- Use `ChatOpenAI()` for multi-turn chat applications that process messages in a conversational format.

In [7]:
llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)
# llm

C:\Users\ASUS K513EQ\AppData\Local\Temp\ipykernel_14724\3166118353.py:1: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  llm=ChatOpenAI(openai_api_key=KEY, model_name="gpt-3.5-turbo", temperature=0.5)


#### Now that the initial requirements are set, let's design the prompt and implement the corresponding chains.

In [8]:
TEMPLATE_QUIZ = """
Input Text: {text}
Given the piece of text above, I would like to ask you to create a quiz of {number} mutiple-choice \
questions for {area} participants. It is necessary that the questions are suitable for {level} level \
and not repeated. It is also important that the response is as in JSON format below.
### JSON RESPONSE
{response_json}
"""

In [9]:
# quiz/response template in nthe form of a dictionary
RESPONSE = {
    "1": {
        "mcq": "question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4"
        },
        "correct_answer": "answer"
    },
    "2": {
        "mcq": "question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4"
        },
        "correct_answer": "answer"
    },
    "3": {
        "mcq": "question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4"
        },
        "correct_answer": "answer"
    },
    "4": {
        "mcq": "question",
        "options": {
            "a": "choice 1",
            "b": "choice 2",
            "c": "choice 3",
            "d": "choice 4"
        },
        "correct_answer": "answer"
    }
}

In [10]:
# prompt
q_gen_prompt = PromptTemplate(
    input_variables=["text", "number", "area", "level", "response_json"],
    template=TEMPLATE_QUIZ
)

In [11]:
quiz_request = LLMChain(llm=llm, prompt=q_gen_prompt, output_key="quiz", verbose=True)

C:\Users\ASUS K513EQ\AppData\Local\Temp\ipykernel_14724\140734323.py:1: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  quiz_request = LLMChain(llm=llm, prompt=q_gen_prompt, output_key="quiz", verbose=True)


In [12]:
TEMPLATE_EVAL = """
Now as a professional, given the multiple-choice quiz for {area} participants, evaluate the quiz \
and provide a thorough analysis of it. Be aware that you can only use 45 words maximum. If necessary, \
update the quiz questions which need to be changed so that it fits student's capabilities.
QUIZ:
{quiz}

"""

In [13]:
# evaluation prompt
quiz_eval_prompt = PromptTemplate(
    input_variables=["area", "quiz",],
    template=TEMPLATE_EVAL
)

In [14]:
eval_request = LLMChain(llm=llm, prompt=quiz_eval_prompt, output_key="evaluation", verbose=True)

In [15]:
# join the chains
gen_eval_chain = SequentialChain(
    chains=[quiz_request, eval_request],
    input_variables=["text", "number", "area", "level", "response_json"],
    output_variables=["quiz", "evaluation"],
    verbose=True
)

In [16]:
# declare the relative path to the data file
data_path = "../data.txt"

In [17]:
# read data
with open(data_path, "r") as file:
    TEXT = file.read()

# print(TEXT)

In order to be able to properly use the quiz template we have already created, we have to first transform it into a JSON-formatted string.

#### How is a JSON String Different from a Python Dictionary?

| Feature            | JSON-Formatted String                           | Python Dictionary                            |
|--------------------|--------------------------------|--------------------------------|
| **Type**          | `str` (string)                | `dict` (dictionary)           |
| **Format**        | Uses **double quotes (`""`)** for keys and values (if string) | Uses **single or double quotes (`''` or `""`)** for keys/values |
| **Data Type Support** | Limited to basic types: `string`, `number`, `boolean`, `null`, `array`, `object` | Supports all Python data types, including tuples, sets, etc. |
| **Usage**         | Mainly used for **data exchange (e.g., APIs, config files)** | Used for **internal Python operations** |
| **Example**       | `'{"name": "Alice", "age": 25}'` | `{"name": "Alice", "age": 25}` |


In [19]:
# transfrom the quiz template into a JSON-fromatted string
# APIs require JSON strings rather than Python dicts
json.dumps(RESPONSE)

'{"1": {"mcq": "question", "options": {"a": "choice 1", "b": "choice 2", "c": "choice 3", "d": "choice 4"}, "correct_answer": "answer"}, "2": {"mcq": "question", "options": {"a": "choice 1", "b": "choice 2", "c": "choice 3", "d": "choice 4"}, "correct_answer": "answer"}, "3": {"mcq": "question", "options": {"a": "choice 1", "b": "choice 2", "c": "choice 3", "d": "choice 4"}, "correct_answer": "answer"}, "4": {"mcq": "question", "options": {"a": "choice 1", "b": "choice 2", "c": "choice 3", "d": "choice 4"}, "correct_answer": "answer"}}'

In [21]:
NUMBER = 4
AREA = "Deep Learning"
LEVEL= "easy"

In [22]:
# set up token usage tracking in Langchain
with get_openai_callback() as cb:
    response = gen_eval_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "area": AREA,
            "level": LEVEL,
            "response_json": json.dumps(RESPONSE)
        }
    )

C:\Users\ASUS K513EQ\AppData\Local\Temp\ipykernel_14724\1164156174.py:3: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = gen_eval_chain(




> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Input Text: Deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]
Some common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language process

`with get_openai_callback() as cb:` opens a context manager using get_openai_callback(). It tracks or logs OpenAI API usage, such as token consumption or request details. `cb` is the callback object that stores relevant API call information.

Now let us check if the information is collected as expected...

In [23]:
print(f"Total Tokens: {cb.total_tokens}")
print(f"Completion Token: {cb.completion_tokens}")
print(f"Prompt Token: {cb.prompt_tokens}")
print(f"Total Cost: {cb.total_cost}")
print(f"Reasoning Error: {cb.reasoning_tokens}")

Total Tokens: 2058
Completion Token: 416
Prompt Token: 1642
Total Cost: 0.003295
Reasoning Error: 0


In [24]:
response

{'text': 'Deep learning is a subset of machine learning that focuses on utilizing neural networks to perform tasks such as classification, regression, and representation learning. The field takes inspiration from biological neuroscience and is centered around stacking artificial neurons into layers and "training" them to process data. The adjective "deep" refers to the use of multiple layers (ranging from three to several hundred or thousands) in the network. Methods used can be either supervised, semi-supervised or unsupervised.[2]\nSome common deep learning network architectures include fully connected networks, deep belief networks, recurrent neural networks, convolutional neural networks, generative adversarial networks, transformers, and neural radiance fields. These architectures have been applied to fields including computer vision, speech recognition, natural language processing, machine translation, bioinformatics, drug design, medical image analysis, climate science, material

In [ ]:
# get the generated quiz
quiz = response.get("quiz")
json.loads(quiz)

{'1': {'mcq': 'What does the term "deep" in "deep learning" refer to?',
  'options': {'a': 'The complexity of the neural network architecture',
   'b': 'The number of layers in the network',
   'c': 'The type of data being processed',
   'd': 'The speed of the learning process'},
  'correct_answer': 'b'},
 '2': {'mcq': 'Which of the following is NOT a common deep learning network architecture?',
  'options': {'a': 'Convolutional neural networks',
   'b': 'Recurrent neural networks',
   'c': 'Decision trees',
   'd': 'Generative adversarial networks'},
  'correct_answer': 'c'},
 '3': {'mcq': 'What is the main benefit of using deep learning algorithms for unsupervised learning tasks?',
  'options': {'a': 'They require less computational resources',
   'b': 'They can handle only labeled data',
   'c': 'They can be applied to more abundant unlabeled data',
   'd': 'They are faster than supervised learning algorithms'},
  'correct_answer': 'c'},
 '4': {'mcq': 'What is the purpose of a hiera